In [1]:
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate  datasets #xformers
import transformers
import torch
import datasets
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00


## **Create HuggingFace dataset from json**

In [ ]:
import json
json_file_path = 'Policy_QA.json'

# Reading the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)


In [ ]:
import pandas as pd
data_list = [{'input': item['input'], 'output': item['output']} for item in data.values()]
df = pd.DataFrame(data_list)
df

,input,output
0,What are the twin goals of Singapore’s NAIS 2....,One is excellence: Singapore will selectively ...
1,"To achieve the vision and goals, Singapore wil...",System 1: Activity Drivers (Enablers: Industry...
2,What are the 15 Actions that Singapore will un...,(1) Anchor new AI Centres of Excellence (CoEs)...
3,How will Singapore contribute to international...,(1) Anchoring key bilateral relationships with...
4,How will Singapore intensify the promotion of ...,(1) Make available tools that enterprises can ...
...,...,...
89,What is the World Bank's Policy on Access to I...,The policy outlines the World Bank's commitmen...
90,Why does the World Bank restrict access to cer...,The policy includes exceptions to protect sens...
91,How can someone request information from the W...,Individuals can request information through th...
92,What are the types of information classified u...,Information is classified into categories such...


In [ ]:
# !huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
from datasets import Dataset, DatasetDict, load_dataset
# Create a Dataset object from your data
qa_dataset = Dataset.from_pandas(df)

# Optionally, split your dataset into training and testing sets
qa_dataset = DatasetDict({
    'train': qa_dataset.train_test_split(test_size=0.2)['train'],
    'test': qa_dataset.train_test_split(test_size=0.2)['test']
})

# Push your dataset to the Hugging Face Hub
qa_dataset.push_to_hub("jiuyuan/policy_AI")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/416 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jiuyuan/policy_AI/commit/ba692e249e579e1cf9e99203aecda940632be9eb', commit_message='Upload dataset', commit_description='', oid='ba692e249e579e1cf9e99203aecda940632be9eb', pr_url=None, pr_revision=None, pr_num=None)

## **Load dataset from HuggingFace**

In [2]:
# Dataset
data_name = "jiuyuan/policy_AI"
training_data = load_dataset(data_name, split="train")
test_data = load_dataset(data_name, split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/75 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/19 [00:00<?, ? examples/s]

In [3]:
training_data

Dataset({
    features: ['input', 'output'],
    num_rows: 75
})

In [4]:
# Model and tokenizer names
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
refined_model = "llama-2-7b-policyAI"

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

In [5]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['input'])):
        text = f"### Question: {example['input'][i]}\n ### Answer: {example['output'][i]}</s>"
        output_texts.append(text)
    return output_texts


## **Fine tune**

In [6]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,#rank
    bias="none",
    task_type="CAUSAL_LM"
)

batch_size_multiplier = 2
# Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    evaluation_strategy = "epoch",
    num_train_epochs=30//batch_size_multiplier,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1*batch_size_multiplier,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=20//batch_size_multiplier,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    eval_dataset = test_data,
    formatting_func=formatting_prompts_func, #hereeee
    peft_config=peft_parameters,
    # dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params
)

# Training
fine_tuning.train()

# Save Model
fine_tuning.model.save_pretrained(refined_model)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,2.231420
2,2.591000,1.967282
2,2.216600,1.742126
4,1.920700,1.594000
4,1.721600,1.521432
6,1.592200,1.422980
6,1.463200,1.322758
8,1.250800,1.211426
8,1.126000,1.083627
10,1.006400,1.003400


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
fine_tuning.model.save_pretrained("/content/drive/MyDrive/94812/llama-2-7b-policyAI")

In [ ]:
# fine_tuning is the trainer
model_path = "/content/drive/MyDrive/94812/llama-2-7b-policyAI"
fine_tuning.save_model(model_path)
fine_tuning.model.config.save_pretrained(model_path+"config.json")

In [ ]:
# model_path = "/content/drive/MyDrive/94812/llama-2-7b-policyAI"

# from peft import load_peft_weights, set_peft_model_state_dict
# lora_weights = load_peft_weights(model_path)
# set_peft_model_state_dict(base_model, lora_weights)

## **Generations & Evaluation**

In [7]:
df_test_all=test_data.to_pandas()
df_train=training_data.to_pandas()
df_test_all

,input,output
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express..."
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro..."
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult..."
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language..."
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...


In [8]:
df_test_all['output'].apply(lambda x:len(x.split())).describe(), df_train['output'].apply(lambda x:len(x.split())).describe()

(count    19.000000
 mean     40.631579
 std      11.591235
 min      17.000000
 25%      33.000000
 50%      41.000000
 75%      48.500000
 max      61.000000
 Name: output, dtype: float64,
 count      75.000000
 mean       70.000000
 std       135.589205
 min        17.000000
 25%        30.000000
 50%        39.000000
 75%        52.000000
 max      1020.000000
 Name: output, dtype: float64)

In [9]:
from tqdm.notebook import tqdm
import gc
generations =[]
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = f"### Question: {df_test_all['input'][i]}\n ### Answer: </s>"
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  output = fine_tuning.model.generate(**inputs, max_new_tokens=100,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  # print(response)
  # break
  generations.append(response)
  del inputs, output
  gc.collect()
  torch.cuda.empty_cache()

generations

generating...:   0%|          | 0/19 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


['<s> ### Question: Why is international cooperation on AI important?\n ### Answer:</s>s it enables groundbreaking research, boosts economic growth, and helps establish ethical AI use cases, linking capita and talents through international collaboration networks will help ensure that AI reaches its full potential</s>\n### Link: https://www.scientificamerican.com/article/why-international-cooperation-on-ai-is-crucial-and-how-we-can-achieve-it/\n### Key',
 "<s> ### Question: Can you describe the approach taken by the research team in preparing for the interviews with academia and industry experts in the AI assessment?\n ### Answer:</s>s the research team did not explicitly define the types of knowledge and skills to be covered, but rather allowed the interviews to be guided by the experts' own priorities and concerns. This resulted in a comprehensive set of themes and subthemes that captured the broad range of knowledge and skills associated with AI development and use.</s>\n\nIn prepara

In [49]:
df_test_all['fine_tuned_llama_generations']=generations
# df_test_all.to_csv('results.csv')

In [11]:
# Model
raw_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
raw_model.config.use_cache = False
raw_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## **Raw model generations**

In [12]:
from tqdm.notebook import tqdm

import gc
generations_original =[]
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = f"### Question: {df_test_all['input'][i]}\n ### Answer:"
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  # pipline = transformers.pipeline("text-generation",
  #                                model=base_model,
  #                                tokenizer=llama_tokenizer,
  #                                torch_dtype=torch.float16,
  #                                )
  # output = pipeline(prompt,temperature=0.2,
  #                               top_k=50,
  #                               top_p=0.9,
  #                    max_length=500)
  # print(output)
  output = raw_model.generate(**inputs, max_new_tokens=100,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  # print(response)
  # break
  generations_original.append(response)
  del inputs, output
  gc.collect()
  torch.cuda.empty_cache()

generations_original

generating...:   0%|          | 0/19 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


['<s> ### Question: Why is international cooperation on AI important?\n ### Answer: International cooperation on AI is important for several reasons:\n\n1. **Sharing knowledge and expertise**: By working together, countries can pool their resources and expertise to advance the field of AI more quickly and effectively. This can lead to the development of new technologies and applications that benefit society as a whole.\n2. **Addressing ethical and social implications**: AI raises a range of ethical and social implications, such as privacy',
 '<s> ### Question: Can you describe the approach taken by the research team in preparing for the interviews with academia and industry experts in the AI assessment?\n ### Answer: The research team took a multi-faceted approach in preparing for the interviews with academia and industry experts in the AI assessment. Here are some of the key steps we took:\n\n1. Literature Review: We conducted a comprehensive literature review to identify the key them

In [50]:
df_test_all['original_llama_generations']=generations_original
df_test_all

,input,output,fine_tuned_llama_generations,original_llama_generations
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...,<s> ### Question: Why is international coopera...,<s> ### Question: Why is international coopera...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,<s> ### Question: Can you describe the approac...,<s> ### Question: Can you describe the approac...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express...",<s> ### Question: What concerns do critics hav...,<s> ### Question: What concerns do critics hav...
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...,<s> ### Question: How does the European Union ...,<s> ### Question: How does the European Union ...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...,<s> ### Question: What are the Universal Guide...,<s> ### Question: What are the Universal Guide...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,<s> ### Question: What role do whistleblowers ...,<s> ### Question: What role do whistleblowers ...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro...",<s> ### Question: How's UK's approach differen...,<s> ### Question: How's UK's approach differen...
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult...",<s> ### Question: How does Japan's approach to...,<s> ### Question: How does Japan's approach to...
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language...",<s> ### Question: What are foundational AI mod...,<s> ### Question: What are foundational AI mod...
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...,<s> ### Question: What additional measures are...,<s> ### Question: What additional measures are...


In [51]:
def extract_text_between_tags(text):
    # Define the regex pattern to match text between </s> tags

    import re
    pattern = r'</s>(.*?)(</s>|$)'

    # Use re.findall() to find all occurrences
    matches = re.findall(pattern, text)[0][0]

    return matches
df_test_all['original_llama_generations']= df_test_all['original_llama_generations'].apply(lambda x: x.split('Answer:', 1)[-1] if 'Answer:' in x else x)
df_test_all['original_llama_generations']= df_test_all['original_llama_generations'].apply(lambda x: x.replace('</s>', '').replace('<s>', '').replace('</s>s', ''))
df_test_all['fine_tuned_llama_generations']= df_test_all['fine_tuned_llama_generations'].apply(lambda x: extract_text_between_tags(x) if 'Answer:' in x else x)
df_test_all['fine_tuned_llama_generations']= df_test_all['fine_tuned_llama_generations'].apply(lambda x: x.replace('</s>', '').replace('<s>', '').replace('</s>s', ''))
df_test_all

,input,output,fine_tuned_llama_generations,original_llama_generations
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...,"s it enables groundbreaking research, boosts e...",International cooperation on AI is important ...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,s the research team did not explicitly define ...,The research team took a multi-faceted approa...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express...","s Critics, including EU businesses, have expre...",Critics of the EU's AI Act have raised severa...
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...,s the European Union's (EU) first comprehensiv...,The European Union (EU) classifies AI systems...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...,"safety and privacy, transparency and accountab...",EPIC (Electronic Privacy Information Center) ...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,s Whistleblowers and complaints are crucial fo...,Whistleblowers and complaints play a crucial ...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro...","s the UK leaves the EU, it will no longer part...",The UK's approach to data protection is diffe...
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult...","s Japan focuses on a risk-based, agile, and mu...",Japan's approach to AI regulation differs fro...
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language...",s Foundational AI models are highly effective ...,Foundational AI models are the basic building...
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...,suggestions include revising existing export c...,International cooperation on foundational AI ...


In [52]:
df_test_all.to_csv('results.csv')

## **Metrics**

In [53]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install nltk rouge -q
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge


In [54]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
def jaccard_similarity(text1, text2):
    """Calculates the Jaccard Similarity between two texts."""
    # Tokenize both texts
    tokens1 = set(text1.lower().split())
    tokens2 = set(text2.lower().split())

    intersection = tokens1.intersection(tokens2)
    union = tokens1.union(tokens2)
    similarity = len(intersection) / len(union)
    return similarity
def calculate_bleu_rouge(reference, candidate):
    """
    Calculate BLEU and ROUGE scores for a pair of strings.

    :param reference: The reference string (ground truth).
    :param candidate: The candidate string (generated text).
    :return: A dictionary containing BLEU and ROUGE scores.
    """
    # Preprocessing the strings: tokenizing into words
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    # Calculating BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)

    # Calculating ROUGE score
    rouge = Rouge()
    rouge_score = rouge.get_scores(candidate, reference)[0]

    return np.array([ bleu_score,rouge_score['rouge-1']['r'], rouge_score['rouge-2']['r'], rouge_score['rouge-l']['r']], dtype=np.float32)

# Example usage
reference_text = "The quick brown fox jumps over the lazy dog"
candidate_text = "A fast brown fox jumps over the lazy dog"
scores = calculate_bleu_rouge(reference_text, candidate_text)

print(scores)


[0.7259795 0.7777778 0.75      0.7777778]


In [55]:
raw_model_metrics = df_test_all.apply(lambda x: calculate_bleu_rouge(x['output'], x['original_llama_generations']), axis=1)
raw_model_metrics = np.array(raw_model_metrics).mean(-1)
raw_model_metrics

array([0.02019156, 0.281702  , 0.07066377, 0.24327506], dtype=float32)

In [56]:
fine_tuned_model_metrics = df_test_all.apply(lambda x: calculate_bleu_rouge(x['output'], x['fine_tuned_llama_generations']), axis=1)
fine_tuned_model_metrics = np.array(fine_tuned_model_metrics).mean(-1)
fine_tuned_model_metrics

array([0.08856215, 0.34528014, 0.14301327, 0.29787165], dtype=float32)

In [57]:
df_metrics = pd.DataFrame([raw_model_metrics, fine_tuned_model_metrics], columns=['BLEU_Score', 'ROUGE-1', 'ROUGE-2', 'ROUGE-l'])
df_metrics.index  = ['Raw_llama2', 'Fine_tuned_llama2_batch*2']
df_metrics

,BLEU_Score,ROUGE-1,ROUGE-2,ROUGE-l
Raw_llama2,0.020192,0.281702,0.070664,0.243275
Fine_tuned_llama2_batch*2,0.088562,0.345280,0.143013,0.297872


In [58]:
df_test_all.columns

Index(['input', 'output', 'fine_tuned_llama_generations',
       'original_llama_generations'],
      dtype='object')

In [59]:
for i in range(len(df_test_all)):
  row = df_test_all.iloc[i]
  print(i)
  print("label:")
  print(row['output'])
  print("generations:")
  print(row['fine_tuned_llama_generations'])

0
label:
International cooperation on AI is crucial due to the collaborative nature of AI research, innovation, and standardization. It maximizes the advantage of scale, focuses on responsible AI development, reduces barriers to innovation, aligns AI regulation, and addresses global challenges like climate change and pandemic preparedness
generations:
s it enables groundbreaking research, boosts economic growth, and helps establish ethical AI use cases, linking capita and talents through international collaboration networks will help ensure that AI reaches its full potential
1
label:
The research team prepared generic interview protocols for broad categories of interviewees, which were later reviewed by a member of RAND's SRG to ensure alignment with the study goals. These protocols were then used to derive individual interview protocols tailored for specific experts
generations:
s the research team did not explicitly define the types of knowledge and skills to be covered, but rather a

In [63]:
for i in range(len(df_test_all)):
  row = df_test_all.iloc[i]
  print(i)
  print("label:")
  print(row['output'])
  print("generations:")
  print(row['original_llama_generations'])

0
label:
International cooperation on AI is crucial due to the collaborative nature of AI research, innovation, and standardization. It maximizes the advantage of scale, focuses on responsible AI development, reduces barriers to innovation, aligns AI regulation, and addresses global challenges like climate change and pandemic preparedness
generations:
 International cooperation on AI is important for several reasons:

1. **Sharing knowledge and expertise**: By working together, countries can pool their resources and expertise to advance the field of AI more quickly and effectively. This can lead to the development of new technologies and applications that benefit society as a whole.
2. **Addressing ethical and social implications**: AI raises a range of ethical and social implications, such as privacy
1
label:
The research team prepared generic interview protocols for broad categories of interviewees, which were later reviewed by a member of RAND's SRG to ensure alignment with the stud